# import

In [1]:
import pandas as pd
import json
import requests
from pathlib import Path
import numpy as np
import sys
import requests
import json
from matplotlib import pyplot as plt
import seaborn as sns
sns.set_context('talk')
%matplotlib inline


# directories


In [2]:
if sys.platform =='linux':
    path_d = Path('/mnt/d')
else:
    path_d = Path('D://')
data_dir = path_d/'data'/'dpe_full'   

plot_dir = Path('.')/'plot'
plot_dir.mkdir(exist_ok=True,parents=True)

# read exemple

In [82]:
dept = '38'
td011 =pd.read_csv(data_dir/'depts'/dept/'td011_installation_chauffage.csv',dtype=str)
td012 =pd.read_csv(data_dir/'depts'/dept/'td012_generateur_chauffage.csv',dtype=str)
td013 =pd.read_csv(data_dir/'depts'/dept/'td013_installation_ecs.csv',dtype=str)
td014 =pd.read_csv(data_dir/'depts'/dept/'td014_generateur_ecs.csv',dtype=str)
td006 =pd.read_csv(data_dir/'depts'/dept/'td006_batiment.csv',dtype=str)
td001 =pd.read_csv(data_dir/'depts'/dept/'td001_dpe.csv',dtype=str)
td005 =pd.read_csv(data_dir/'depts'/dept/'td005_fiche_technique.csv',dtype=str)

In [83]:
import pandas as pd
import numpy as np
from assets_orm import DPEMetaData
from td001_merge import merge_td001_dpe_id_system

td013_types = {'td013_installation_ecs_id': 'str',
 'td006_batiment_id': 'str',
 'tr005_type_installation_ecs_id': 'str',
 'nombre_appartements_echantillon': 'float',
 'surface_habitable_echantillon': 'float',
 'becs': 'float',
 'tv039_formule_becs_id': 'category',
 'surface_alimentee': 'float',
 'td001_dpe_id': 'str'}


td014_types = {'td014_generateur_ecs_id': 'str',
 'td013_installation_ecs_id': 'str',
 'tr004_type_energie_id': 'str',
 'tv045_conversion_kwh_co2_id': 'category',
 'tv046_evaluation_contenu_co2_reseaux_id': 'category',
 'coefficient_performance': 'float',
 'tv032_coefficient_performance_id': 'category',
 'rpn': 'float',
 'qp0': 'float',
 'puissance_veilleuse': 'float',
 'tv036_puissance_veilleuse_id': 'category',
 'tv037_puissance_necessaire_production_ecs_id': 'category',
 'puissance_nominale': 'float',
 'tv038_puissance_nominale_id': 'category',
 'tv040_rendement_distribution_ecs_id': 'category',
 'tv047_rendement_generation_ecs_id': 'category',
 'volume_stockage': 'float',
 'tv048_rendement_stockage_ecs_id': 'category',
 'tv049_perte_stockage_ecs_id': 'category',
 'tv041_coefficient_emplacement_fonctionnement_id': 'category',
 'tv043_pertes_stockage_id': 'category',
 'tv019_fecs_id': 'category',
 'tv027_pertes_recuperees_ecs_id': 'category',
 'td001_dpe_id': 'str'}

def merge_td013_tr_tv(td013):
    meta = DPEMetaData()
    table = td013.copy()
    table = meta.merge_all_tr_table(table)
    table = meta.merge_all_tv_table(table)
    table = table.astype(td013_types)
    table = table.rename(columns={'id': 'td013_installation_ecs_id'})

    return table

def merge_td014_tr_tv(td014):
    meta = DPEMetaData()
    table = td014.copy()
    table = meta.merge_all_tr_table(table)
    table = meta.merge_all_tv_table(table)
    table = table.astype(td014_types)
    table = table.rename(columns={'id': 'td013_installation_ecs_id'})

    return table

td001,td006,td011, td012,td013,td014 = merge_td001_dpe_id_system(td001, td006, td011, td012,td013,td014)

td013 = merge_td013_tr_tv(td013)

td014 = merge_td014_tr_tv(td014)

In [52]:
from utils import strip_accents,affect_lib_by_matching_score

In [73]:
td005_ecs=td005.loc[td005.tr011_sous_categorie_fiche_technique_id=='17']
vr_ecs = td005_ecs.valeur_renseignee.str.lower().apply(lambda x:strip_accents(x))



In [102]:
td012.loc[td012.td001_dpe_id=='2151372'].coefficient_performance

26841    2.20
26842    1.00
Name: coefficient_performance, dtype: object

In [103]:
libs=vr_ecs.apply(lambda x:affect_lib_by_matching_score(x,gen_ecs_normalized_lib_matching_dict_ft))

KeyboardInterrupt: 

In [130]:
test=libs=='pompe a chaleur combinee ecs/chauffage (indeterminee)'
test=libs=='pompe a chaleur combinee ecs/chauffage (pac air/air)'
test=libs=='ballon electrique'
test=libs=='chaudiere mixte gaz'
test=libs=='non affecte'
test=libs=='ecs collective reseau de chaleur'
test=libs=='chaudiere gaz'
test=libs=='ballon thermodynamique electrique'
test=libs=='chaudiere fioul'
test=libs=='chauffe-eau fioul independant'

#test=libs=='ecs collective reseau de chaleur'
#test=libs=='ecs collective reseau de chaleur'
#test=libs=='ecs collective reseau de chaleur'

In [131]:
libs.value_counts()

ballon electrique                                            71480
chaudiere mixte gaz                                          26853
non affecte                                                  26576
chaudiere mixte fioul                                         9057
ecs collective reseau de chaleur                              7999
chaudiere gaz                                                 5195
ballon thermodynamique electrique                             4985
chaudiere fioul                                               1801
chaudiere mixte bois                                          1729
chauffe-eau gaz independant                                   1207
pompe a chaleur combinee ecs/chauffage (pac air/eau)          1117
ecs instantanee electrique                                     780
pompe a chaleur combinee ecs/chauffage (indeterminee)          137
pompe a chaleur combinee ecs/chauffage (pac geothermique)      113
pompe a chaleur combinee ecs/chauffage (pac air/air)          

In [132]:
vr_ecs.loc[test].head(100).to_dict()

{859283: ' - production ecs par chauffe bains instantane avec veilleuse\\n\\nproduction dans le volume habitable\\nles pieces desservies sont non contigues\\n\\n\\n - production ecs par chauffe bains instantane\\n\\nproduction collective en releve de factures',
 885390: "ecs n°1 \\n type d'installation : sans ecs solaire \\n energie : fioul \\n type equipement : chauffe bain gaz \\n contiguite des pieces : pieces contigues \\n position de la production : en volume chauffe \\n isolation du reseau : reseau isole \\n volume de stockage : 10 \\n veilleuse : non \\n alimentation : heure pleine \\n ",
 2107176: 'ecs par chaudiere fioul. 1 ballon horizontal ancien, 1 ballon vertical calorifuge recent<br/>',
 2991930: ' - production ecs par chauffe bains instantane\\n - production ecs par chauffe bains instantane\\n\\nproduction collective en releve de factures'}

In [129]:
td005_ecs.loc[test].iloc[5]

id                                                                                  34876246
td001_dpe_id                                                                         1986629
tr011_sous_categorie_fiche_technique_id                                                   17
valeur_renseignee                          Installation d'ECS 90 m²\r\n- Ecs 1 : chauffe-...
est_efface                                                                                 0
Name: 661298, dtype: object

In [77]:
td012

,id,systeme_chauffage_cogeneration_id,td011_installation_chauffage_id,tr004_type_energie_id,tv045_conversion_kwh_co2_id,tv046_evaluation_contenu_co2_reseaux_id,rendement_emission_systeme_chauffage,tv028_rendement_emission_systeme_chauffage_id,rendement_distribution_systeme_chauffage,tv029_rendement_distribution_systeme_chauffage_id,...,tv034_temperature_fonctionnement_chaudiere_100_id,tv035_temperature_fonctionnement_chaudiere_30_id,rpn,rpint,qp0,puissance_veilleuse,tv036_puissance_veilleuse_id,puissance_nominale,tv038_puissance_nominale_id,consommation_chauffage
0,263,NaN,233,2,8,NaN,0.95,NaN,1.00,NaN,...,NaN,NaN,0.00,0.00,0.00,0.00,NaN,0.00,NaN,7425.01
1,470,NaN,388,2,8,NaN,0.97,NaN,1.00,NaN,...,NaN,NaN,0.00,0.00,0.00,0.00,NaN,0.00,NaN,3844.48
2,471,NaN,389,1,1,NaN,0.95,NaN,1.00,NaN,...,NaN,NaN,0.00,0.00,0.00,0.00,NaN,0.00,NaN,6070.93
3,620,NaN,530,4,3,NaN,0.95,NaN,0.91,NaN,...,NaN,NaN,86.89,84.34,0.28,0.00,NaN,28.00,NaN,37863.52
4,1285,NaN,1119,4,3,NaN,0.95,NaN,0.91,NaN,...,NaN,NaN,82.51,81.77,0.18,0.00,NaN,18.00,NaN,14408.24
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
108095,5364279,NaN,4954352,1,1,NaN,0.95,NaN,1.00,NaN,...,NaN,NaN,0.00,0.00,0.00,0.00,NaN,0.00,NaN,25983.22
108096,5364357,NaN,4954427,3,2,NaN,0.95,NaN,0.91,NaN,...,NaN,NaN,86.76,84.14,0.24,0.00,NaN,24.00,NaN,4350.46
108097,5364464,NaN,4954526,3,2,NaN,0.95,NaN,1.00,NaN,...,NaN,NaN,73.00,73.00,0.00,0.00,NaN,18.00,NaN,4001.35
108098,5364521,NaN,4954579,3,2,NaN,0.95,NaN,0.91,NaN,...,NaN,NaN,86.76,84.14,0.24,0.00,NaN,24.00,NaN,10024.54


In [57]:
unique_gen_ecs = vr_ecs.unique()
libs = {k: affect_lib_by_matching_score(k, gen_ecs_normalized_lib_matching_dict_ft) for k in
                         unique_gen_ecs}


NameError: name 'table' is not defined

In [58]:
gen_ecs_lib_infer_ft = vr_ecs.replace(libs)

KeyboardInterrupt: 

In [ ]:
gen_ecs_lib_infer_ft.value_counts()

In [59]:
len(libs)

70996

In [45]:
is_mixte = vr_ecs.str.contains('mixte')|vr_ecs.str.contains('combine')
is_reseau = vr_ecs.str.contains('reseau de chaleur')
is_chaudiere = vr_ecs.str.contains('chaudiere')
is_ballon_elec = vr_ecs.str.contains('lectrique')
is_pac = vr_ecs.str.contains('pompe a chaleur') | vr_ecs.str.contains('pac')

is_thermodynamique= vr_ecs.str.contains('thermodynamique')

is_pac_comb = is_mixte | is_pac





In [ ]:
'ballon electrique'


In [49]:
vr_ecs.loc[is_thermodynamique].iloc[3]

"chauffe-eau thermodynamique sur geothermie, hors volume de chauffe, reseau d'eau isole<br/>becs : 2099, rd : 1, rg : 1.7, pn : 44, iecs : 0.68, fecs : 0"

In [23]:
vr_ecs.value_counts()

 - production ecs par générateur mixte (chauffage et ecs)                                                                                                                                                                                                                    5735
 - raccordement à un réseau de chaleur                                                                                                                                                                                                                                       1971
type d'équipement : \nenergie : \ntype d'installation : individuelle, sans solaire\ndistribution : production dans le volume chauffé, pièces alimentées contiguës\n : \n                                                                                                     1928
 - production ecs par générateur mixte (chauffage et ecs)\n production assurée par la génération : génération n°1 ( générateur 1 )\ninstallation individuelle production instantan

In [79]:
import pandas as pd
import numpy as np
from assets_orm import DPEMetaData
from td001_merge import merge_td001_dpe_id_system

td013_types = {'td013_installation_ecs_id': 'str',
 'td006_batiment_id': 'str',
 'tr005_type_installation_ecs_id': 'str',
 'nombre_appartements_echantillon': 'float',
 'surface_habitable_echantillon': 'float',
 'becs': 'float',
 'tv039_formule_becs_id': 'category',
 'surface_alimentee': 'float',
 'td001_dpe_id': 'str'}


td014_types = {'td014_generateur_ecs_id': 'str',
 'td013_installation_ecs_id': 'str',
 'tr004_type_energie_id': 'str',
 'tv045_conversion_kwh_co2_id': 'category',
 'tv046_evaluation_contenu_co2_reseaux_id': 'category',
 'coefficient_performance': 'float',
 'tv032_coefficient_performance_id': 'category',
 'rpn': 'float',
 'qp0': 'float',
 'puissance_veilleuse': 'float',
 'tv036_puissance_veilleuse_id': 'category',
 'tv037_puissance_necessaire_production_ecs_id': 'category',
 'puissance_nominale': 'float',
 'tv038_puissance_nominale_id': 'category',
 'tv040_rendement_distribution_ecs_id': 'category',
 'tv047_rendement_generation_ecs_id': 'category',
 'volume_stockage': 'float',
 'tv048_rendement_stockage_ecs_id': 'category',
 'tv049_perte_stockage_ecs_id': 'category',
 'tv041_coefficient_emplacement_fonctionnement_id': 'category',
 'tv043_pertes_stockage_id': 'category',
 'tv019_fecs_id': 'category',
 'tv027_pertes_recuperees_ecs_id': 'category',
 'td001_dpe_id': 'str'}

def merge_td013_tr_tv(td013):
    meta = DPEMetaData()
    table = td013.copy()
    table = meta.merge_all_tr_table(table)
    table = meta.merge_all_tv_table(table)
    table = table.astype(td013_types)
    table = table.rename(columns={'id': 'td013_installation_ecs_id'})

    return table

def merge_td014_tr_tv(td014):
    meta = DPEMetaData()
    table = td014.copy()
    table = meta.merge_all_tr_table(table)
    table = meta.merge_all_tv_table(table)
    table = table.astype(td014_types)
    table = table.rename(columns={'id': 'td013_installation_ecs_id'})

    return table

td001,td006,td011, td012,td013,td014 = merge_td001_dpe_id_system(td001, td006, td011, td012,td013,td014)

td013 = merge_td013_tr_tv(td013)

td014 = merge_td014_tr_tv(td014)

In [42]:
td014.count().sort_values(ascending=False).to_dict()

{'td014_generateur_ecs_id': 109506,
 'td001_dpe_id': 109506,
 'tr004_est_efface': 109506,
 'tr004_simulateur': 109506,
 'puissance_nominale': 109506,
 'tr004_description': 109506,
 'tv045_conversion_kwh_co2_id': 109506,
 'tr004_type_energie_id': 109506,
 'td013_installation_ecs_id': 109506,
 'volume_stockage': 109302,
 'rpn': 107352,
 'qp0': 107352,
 'coefficient_performance': 107352,
 'puissance_veilleuse': 107352,
 'tv045_Type de production': 105074,
 'tv045_Méthode': 105074,
 'tv045_Energie': 105074,
 'tv045_Taux de conversion\nkg/kWh': 105074,
 "tv048_Type d'accumulation": 32717,
 'tv040_Type de générateur': 32717,
 'tv040_rendement_distribution_ecs_id': 32717,
 'tv048_rendement_stockage_ecs_id': 32717,
 "tv040_Type d'installation": 32717,
 'tv048_Rs': 32717,
 'tv040_Rd': 32717,
 'tv049_Type de stockage': 31582,
 'tv049_Qg,w': 31582,
 'tv049_perte_stockage_ecs_id': 31582,
 'tv047_Rg': 30012,
 'tv047_rendement_generation_ecs_id': 30012,
 'tv047_Type de Générateur': 30012,
 'tv040_Pr

In [30]:
table = td014

In [86]:
table.columns

Index(['td014_generateur_ecs_id', 'td013_installation_ecs_id',
       'tr004_type_energie_id', 'tv045_conversion_kwh_co2_id',
       'tv046_evaluation_contenu_co2_reseaux_id', 'coefficient_performance',
       'tv032_coefficient_performance_id', 'rpn', 'qp0', 'puissance_veilleuse',
       'tv036_puissance_veilleuse_id',
       'tv037_puissance_necessaire_production_ecs_id', 'puissance_nominale',
       'tv038_puissance_nominale_id', 'tv040_rendement_distribution_ecs_id',
       'tv047_rendement_generation_ecs_id', 'volume_stockage',
       'tv048_rendement_stockage_ecs_id', 'tv049_perte_stockage_ecs_id',
       'tv041_coefficient_emplacement_fonctionnement_id',
       'tv043_pertes_stockage_id', 'tv019_fecs_id',
       'tv027_pertes_recuperees_ecs_id', 'td001_dpe_id', 'tr004_description',
       'tr004_est_efface', 'tr004_simu_ordre', 'tr004_simulateur',
       'tv045_Energie', 'tv045_Méthode', 'tv045_Type de production',
       'tv045_Taux de conversion\nkg/kWh', 'tv046_Département',


In [87]:
is_chaudiere = table.rpn>0


gen_ecs_concat_txt_desc = table["tv027_Type d'installation"].astype('string').replace(np.nan, '') + ' '
gen_ecs_concat_txt_desc.loc[is_chaudiere] += 'chaudiere '

gen_ecs_concat_txt_desc += table['tv027_Type de système'].astype('string').replace(np.nan, '') + ' '
gen_ecs_concat_txt_desc += table["tv032_Type de Générateur"].astype('string').replace(np.nan, '') + ' '
gen_ecs_concat_txt_desc += table['tv036_Type de génération'].astype('string').replace(np.nan, '') + ' '
gen_ecs_concat_txt_desc += table['tv037_Type de Production'].astype('string').replace(np.nan, '') + ' '
gen_ecs_concat_txt_desc += table['tv040_Type de générateur'].astype('string').replace(np.nan, '') + ' '
gen_ecs_concat_txt_desc += table["tv040_Type d'installation"].astype('string').replace(np.nan, '') + ' '
gen_ecs_concat_txt_desc += table["tr004_description"].astype('string').replace(np.nan, '') + ' '
gen_ecs_concat_txt_desc += table["tv045_Energie"].astype('string').replace(np.nan, '') + ' '
gen_ecs_concat_txt_desc += table['tv047_Type de Générateur'].astype('string').replace(np.nan, '') + ' '

In [40]:
gen_ecs_normalized_lib_matching_dict = {
    "ballon thermodynamique electrique": [('individuelle ballon', 'chauffe-eau'), ('pac','thermodynamique', 'air extrait','air exterieur'),
                                          ('electricite', 'electrique')],
    "ballon electrique": [('individuelle ballon', 'chauffe-eau','classique'), ('electricite', 'electrique')],
    "ecs instantanee electrique": [( 'instantanee', 'chauffe-eau'), ('electricite', 'electrique')],

    'chaudiere mixte gaz': ["chaudiere", 'mixte', ("gaz", "gpl")],
    'chaudiere mixte fioul': ["chaudiere", "mixte", "fioul"],
    'chaudiere mixte bois': [("combustion mixte", "chaudiere mixte"), ("classe", "chauffage + ecs"),
                             ("bois", "biomasse")],

    'chauffe-eau gaz independant': [("individuelle ballon", "chauffe-eau", "accumulateur", "chauffe bain"), "gaz"],
    'chauffe-eau gpl independant': [("individuelle ballon", "chauffe-eau", "accumulateur", "chauffe bain"), "gpl"],

    'chauffe-eau fioul independant': [("individuelle ballon", "chauffe-eau", "accumulateur", "chauffe bain"), "fioul"],
    "ecs collective reseau chaleur": ["reseau", "chaleur"],

    'chaudiere gaz': ["chaudiere", "gaz"],
    'chaudiere gpl': ["chaudiere", "gpl"],

    'chaudiere fioul': ["chaudiere", "fioul"],

}

In [89]:
from utils import clean_str,strip_accents,affect_lib_by_matching_score

In [90]:
    gen_ecs_concat_txt_desc = gen_ecs_concat_txt_desc.str.lower().apply(lambda x: strip_accents(x))

    table['gen_ecs_concat_txt_desc'] = gen_ecs_concat_txt_desc

    table['gen_ecs_concat_txt_desc'] = table['gen_ecs_concat_txt_desc'].apply(lambda x: clean_str(x))

    # calcul gen_ecs_lib_infer par matching score text.
    unique_gen_ecs = table.gen_ecs_concat_txt_desc.unique()
    gen_ecs_lib_infer_dict = {k: affect_lib_by_matching_score(k, gen_ecs_normalized_lib_matching_dict) for k in
                             unique_gen_ecs}
    table['gen_ecs_lib_infer'] = table.gen_ecs_concat_txt_desc.replace(gen_ecs_lib_infer_dict)
    is_pac = table.coefficient_performance>2
    table.loc[is_pac,'gen_ecs_lib_infer']='ballon thermodynamique electrique' 

In [91]:
table['gen_ecs_lib_infer'].value_counts()

ballon electrique                    60020
chaudiere gaz                        25580
chaudiere mixte gaz                   7434
non affecte                           5987
ballon thermodynamique electrique     5039
chaudiere fioul                       3218
chaudiere mixte fioul                 1380
ecs collective reseau chaleur          719
ecs instantanee electrique              67
chauffe-eau gaz independant             58
chauffe-eau fioul independant            4
Name: gen_ecs_lib_infer, dtype: int64

In [68]:
table.loc[is_elec].gen_ecs_concat_txt_desc.value_counts()

electricite electricite (hors electricite d'origine renouvelable utilisee dans le batiment)                                                                                                                                          43416
individuelle ballon en volume habitable electrique classique individuelle electricite electricite (hors electricite d'origine renouvelable utilisee dans le batiment) electrique                                                      7336
individuelle ballon hors volume habitable electrique classique individuelle electricite electricite (hors electricite d'origine renouvelable utilisee dans le batiment) electrique                                                    2778
electrique classique individuelle electricite electricite (hors electricite d'origine renouvelable utilisee dans le batiment) electrique                                                                                              2263
semi-accumulation electrique classique individuelle electric

In [65]:
table.loc[is_elec].count().sort_values(ascending=False).to_dict()

{'gen_ecs_lib_infer': 62296,
 'gen_ecs_concat_txt_desc': 62296,
 'td013_installation_ecs_id': 62296,
 'tr004_type_energie_id': 62296,
 'tv045_conversion_kwh_co2_id': 62296,
 'puissance_nominale': 62296,
 'td001_dpe_id': 62296,
 'tr004_description': 62296,
 'tr004_est_efface': 62296,
 'tr004_simulateur': 62296,
 'td014_generateur_ecs_id': 62296,
 'volume_stockage': 62288,
 'tv045_Energie': 62245,
 'tv045_Méthode': 62245,
 'tv045_Type de production': 62245,
 'tv045_Taux de conversion\nkg/kWh': 62245,
 'coefficient_performance': 60811,
 'puissance_veilleuse': 60811,
 'qp0': 60811,
 'rpn': 60811,
 "tv040_Type d'installation": 18803,
 'tv040_Type de générateur': 18803,
 'tv048_rendement_stockage_ecs_id': 18803,
 'tv040_rendement_distribution_ecs_id': 18803,
 'tv040_Rd': 18803,
 'tv048_Rs': 18803,
 "tv048_Type d'accumulation": 18803,
 'tv049_perte_stockage_ecs_id': 18788,
 'tv049_Type de stockage': 18788,
 'tv049_Qg,w': 18788,
 'tv040_Production en volume habitable': 18177,
 'tv043_pertes_st